In [2]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [3]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [4]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [39]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [40]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [ ]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

In [ ]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

In [ ]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df

In [ ]:
course_data_records = course_data_df.to_dict('records')
course_data_records

In [ ]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [ ]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

In [ ]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

In [ ]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

In [ ]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [ ]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

In [ ]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


In [ ]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

## Insert `Class` Records into Salesforce

In [ ]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Insert Staff Data into Saleforce

In [27]:
#Querry MySQL and transorm data
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [28]:
staff_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c', 
    'BirthDate' : 'BirthDate__c'
}, inplace=True)
staff_data_df.head()

,ID_Staff,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [29]:
#Select columns to be exported to SF
staff_data_df.drop(columns = ['ID_Staff'], inplace=True)
staff_data_df['BirthDate__c'] = pd.to_datetime(staff_data_df['BirthDate__c']).dt.date
staff_data_df

,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [31]:
#convert dataframe in a dictionary for SF 
staff_data_records = staff_data_df.to_dict('records')
staff_data_records


[{'EmployeeID__c': '000184220',
  'LastName__c': 'Luongo',
  'FirstName__c': 'Darick',
  'MiddleName__c': 'Nico',
  'BirthDate__c': datetime.date(1995, 7, 5)},
 {'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': datetime.date(1992, 4, 22)},
 {'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': datetime.date(1993, 5, 21)},
 {'EmployeeID__c': '000159108',
  'LastName__c': 'Rodney',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Nicole',
  'BirthDate__c': datetime.date(1994, 6, 7)}]

In [33]:
#Export data (dict) to SF
for rec in staff_data_records:

    record = {
        'EmployeeID__c':rec['EmployeeID__c'],
        'LastName__c':rec['LastName__c'],
        'FirstName__c':rec['FirstName__c'],
        'MiddleName__c':rec['MiddleName__c'], 
        'BirthDate__c' : str(rec['BirthDate__c'])
       }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

## Staff Assignment Table
### Create staff look up table

In [196]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT EmployeeID__c, Name FROM staff__c")
for row in data:
    rec = {
        'ID_Staff__c': row['Name'], # this is a critical line of code
        'EmployeeID__c': row['EmployeeID__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'ID_Staff__c': 'a024x000002hCVT', 'EmployeeID__c': '000160655'},
 {'ID_Staff__c': 'a024x000002hCVO', 'EmployeeID__c': '000130109'},
 {'ID_Staff__c': 'a024x000002hCVJ', 'EmployeeID__c': '000184220'},
 {'ID_Staff__c': 'a024x000002hCVY', 'EmployeeID__c': '000159108'}]

In [197]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_Staff__c,EmployeeID__c
0,a024x000002hCVT,000160655
1,a024x000002hCVO,000130109
2,a024x000002hCVJ,000184220
3,a024x000002hCVY,000159108


### Create class look up table

In [32]:
sf_class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 

query ='SELECT ID_Course__c, Name, Section__c FROM Class__c'

data = sf.query_all_iter(query)
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
        
    }
    sf_class_lookup_list.append(rec)
    
for rec in sf_class_lookup_list:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3] 
    
sf_class_lookup_list

[{'ID_Class__c': 'a014x000008VgEY',
  'ID_Course__c': 'a004x000003VBnx',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008VgEi',
  'ID_Course__c': 'a004x000003VBo7',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008VgEd',
  'ID_Course__c': 'a004x000003VBo2',
  'Section__c': 'GWARL201905WEB3'},
 {'ID_Class__c': 'a014x000008VgET',
  'ID_Course__c': 'a004x000003VBnx',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'}]

In [33]:
pd.DataFrame(data)

""


In [34]:
sf_class_lookup_df = pd.DataFrame(sf_class_lookup_list)
sf_class_lookup_df

,ID_Class__c,ID_Course__c,Section__c
0,a014x000008VgEY,a004x000003VBnx,GWDC201805DATA3
1,a014x000008VgEi,a004x000003VBo7,GWARL201905UIUX3
2,a014x000008VgEd,a004x000003VBo2,GWARL201905WEB3
3,a014x000008VgET,a004x000003VBnx,GWU-ARL-DATA-PT-09-0


In [24]:
sf_course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 

query ='''
    SELECT
        Course_Code__c,
        Name
        FROM Course__c
    '''

data = sf.query_all_iter(query)
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
       
        
    }
    sf_course_lookup_list.append(rec)
    
print(sf_course_lookup_list)

sf_course_lookup_df = pd.DataFrame(sf_course_lookup_list)
sf_course_lookup_df


[{'ID_Course__c': 'a004x000003VBo2', 'Course_Code__c': 'BC-WEBDEV'}, {'ID_Course__c': 'a004x000003VBo7', 'Course_Code__c': 'BC-UIUX'}, {'ID_Course__c': 'a004x000003VBoM', 'Course_Code__c': 'CIS-438'}, {'ID_Course__c': 'a004x000003VBoR', 'Course_Code__c': 'CIS-430'}, {'ID_Course__c': 'a004x000003VBoW', 'Course_Code__c': 'CIS-435'}, {'ID_Course__c': 'a004x000003VBnx', 'Course_Code__c': 'BC-DATAVIZ'}, {'ID_Course__c': 'a004x000003VBoH', 'Course_Code__c': 'CIS-405'}, {'ID_Course__c': 'a004x000003VBoC', 'Course_Code__c': 'CIS-349'}]


,ID_Course__c,Course_Code__c
0,a004x000003VBo2,BC-WEBDEV
1,a004x000003VBo7,BC-UIUX
2,a004x000003VBoM,CIS-438
3,a004x000003VBoR,CIS-430
4,a004x000003VBoW,CIS-435
5,a004x000003VBnx,BC-DATAVIZ
6,a004x000003VBoH,CIS-405
7,a004x000003VBoC,CIS-349


In [36]:
# Merge class and course Salesforce tables to create a natural key class lookup table

class_lookup_table = pd.merge(sf_course_lookup_df, sf_class_lookup_df)
class_lookup_table

,ID_Course__c,Course_Code__c,ID_Class__c,Section__c
0,a004x000003VBo2,BC-WEBDEV,a014x000008VgEd,GWARL201905WEB3
1,a004x000003VBo7,BC-UIUX,a014x000008VgEi,GWARL201905UIUX3
2,a004x000003VBnx,BC-DATAVIZ,a014x000008VgEY,GWDC201805DATA3
3,a004x000003VBnx,BC-DATAVIZ,a014x000008VgET,GWU-ARL-DATA-PT-09-0


In [42]:
# Query the `staffassignment` table, staff and class table from MySQL
# extend the class table query to the course table o establish natural key records.
query = '''
    SELECT 
        sf.EmployeeID As 'EmployeeID__c',
        sa.Role AS Role__c,
        sa.StartDate AS Start_Date__c,
        sa.EndDate AS EndDate__c,
        cl.Section AS Section__c,
        co.CourseCode AS CourseCode__c       
    FROM 
        staff sf
        INNER JOIN staffassignment sa
        ON sf.ID_Staff = sa.ID_Staff
        INNER JOIN class cl
        ON sa.ID_Class = cl.ID_class
        INNER JOIN course co
        on cl.ID_Course = co.ID_course
'''

staffassign_data_df = pd.read_sql(query, conn)
staffassign_data_df.head() 

,EmployeeID__c,Role__c,Start_Date__c,EndDate__c,Section__c,CourseCode__c
0,000184220,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,000159108,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,000160655,Instructor,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [219]:
staffassign_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'Role':'Role__c',
    'ID_Class': 'ID_Class__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
}, inplace=True)
staffassign_data_df.head()

,EmployeeID__c,Role__c,ID_Class__c,Start_Date__c,End_Date__c
0,000184220,Teacher Assistant,1,2020-09-16,None
1,000159108,Teacher Assistant,1,2020-09-16,None
2,000160655,Instructor,1,2020-09-16,None


In [220]:
## Merge student df with student lookup
staffassign_data_df_mg1 = pd.merge(staffassign_data_df, staff_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

staffassign_data_df_mg1.head()

,EmployeeID__c,Role__c,ID_Class__c,Start_Date__c,End_Date__c,ID_Staff__c
0,000184220,Teacher Assistant,1,2020-09-16,None,a024x000002hCVJ
1,000159108,Teacher Assistant,1,2020-09-16,None,a024x000002hCVY
2,000160655,Instructor,1,2020-09-16,None,a024x000002hCVT


In [221]:
## Merge classpart_data_df_mg1 df with class lookup
staffassign_data_df_mg2 = pd.merge(staffassign_data_df_mg1, class_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

staffassign_data_df_mg2.head()

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
classparticipant = classpart_data_df_mg2[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]
#classparticipant['Start_Date__c'] = pd.to_datetime(classparticipant['Start_Date__c']).dt.date
classparticipant.head()

## Insert Student Records into Salesforce

In [122]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [123]:
student_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate' : 'BirthDate__c',
    'Gender' : 'Gender__c'
}, inplace=True)
student_data_df.head()

,ID_Student,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [124]:
student_data_df.drop(columns = ['ID_Student'], inplace=True)
#student_data_df['BirthDate__c'] = student_data_df['BirthDate__c'].replace({None: "Null"})
#student_data_df['BirthDate__c'] = pd.to_datetime(student_data_df['BirthDate__c']).dt.date

student_data_df.head()

,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [125]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Precious',
  'MiddleName__c': 'Mariette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '

In [126]:
for rec in student_data_records:

    record = {
        'StudentID__c':rec['StudentID__c'],
        'LastName__c':rec['LastName__c'],
        'FirstName__c':rec['FirstName__c'],
        'MiddleName__c':rec['MiddleName__c'],
        'BirthDate__c' : rec['BirthDate__c'],
        'Gender__c' : rec['Gender__c']
    }
    
    try:
        sf.student__c.create(record)
    except Exception as e:
        print(e)

### student lookup table

In [128]:
student_lookup_list = []
data = sf.query_all_iter("SELECT StudentID__c, Name FROM student__c")
for row in data:
    rec = {
        'ID_Student__c': row['Name'], # this is a critical line of code
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)
    
student_lookup_list

[{'ID_Student__c': 'a034x000002jhyX', 'StudentID__c': '25005250'},
 {'ID_Student__c': 'a034x000002jhxU', 'StudentID__c': '25003778'},
 {'ID_Student__c': 'a034x000002jhxZ', 'StudentID__c': '25003605'},
 {'ID_Student__c': 'a034x000002jhxj', 'StudentID__c': '25004152'},
 {'ID_Student__c': 'a034x000002jhx0', 'StudentID__c': '25003514'},
 {'ID_Student__c': 'a034x000002jhxt', 'StudentID__c': '25002311'},
 {'ID_Student__c': 'a034x000002jhz1', 'StudentID__c': '25006027'},
 {'ID_Student__c': 'a034x000002jhy3', 'StudentID__c': '25006805'},
 {'ID_Student__c': 'a034x000002jhy8', 'StudentID__c': '25006947'},
 {'ID_Student__c': 'a034x000002jhyS', 'StudentID__c': '25002555'},
 {'ID_Student__c': 'a034x000002jhyD', 'StudentID__c': '25002714'},
 {'ID_Student__c': 'a034x000002jhxe', 'StudentID__c': '25002056'},
 {'ID_Student__c': 'a034x000002jhxy', 'StudentID__c': '25005602'},
 {'ID_Student__c': 'a034x000002jhyc', 'StudentID__c': '25002325'},
 {'ID_Student__c': 'a034x000002jhzB', 'StudentID__c': '2500733

In [129]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head()

,ID_Student__c,StudentID__c
0,a034x000002jhyX,25005250
1,a034x000002jhxU,25003778
2,a034x000002jhxZ,25003605
3,a034x000002jhxj,25004152
4,a034x000002jhx0,25003514


## Class Lookup Table

In [149]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Section__c, Name FROM class__c")
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'Section__c': row['Section__c']
    }
    class_lookup_list.append(rec)
    
class_lookup_list

[{'ID_Class__c': 'a014x000008VgEY', 'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008VgEi', 'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008VgEd', 'Section__c': 'GWARL201905WEB3'},
 {'ID_Class__c': 'a014x000008VgET', 'Section__c': 'GWU-ARL-DATA-PT-09-0'}]

In [150]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,ID_Class__c,Section__c
0,a014x000008VgEY,GWDC201805DATA3
1,a014x000008VgEi,GWARL201905UIUX3
2,a014x000008VgEd,GWARL201905WEB3
3,a014x000008VgET,GWU-ARL-DATA-PT-09-0


In [170]:
# Query the `Class` and student table from MySQL
query = '''
    SELECT 
        st.StudentID,
        cl.section,
        cp.StartDate,
        cp.EndDate
        
    FROM 
        classparticipant cp
        INNER JOIN student st
        ON cp.ID_Student = st.ID_Student
        INNER JOIN class cl
        ON cl.ID_Class = cp.ID_class

'''

classpart_data_df = pd.read_sql(query, conn)
classpart_data_df.head()

,StudentID,section,StartDate,EndDate
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [171]:
classpart_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)

In [172]:
classpart_data_df.head()

,StudentID__c,Section__c,Start_Date__c,End_Date__c
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [173]:
## Merge student df with student lookup
classpart_data_df_mg1 = pd.merge(classpart_data_df, student_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

classpart_data_df_mg1.head()

,StudentID__c,Section__c,Start_Date__c,End_Date__c,ID_Student__c
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhwv
1,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhx0
2,25005833,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhx5
3,25007334,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhzG
4,25002589,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhxA


In [182]:
## Merge classpart_data_df_mg1 df with class lookup
classpart_data_df_mg2 = pd.merge(classpart_data_df_mg1, class_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

classpart_data_df_mg2.head()

,StudentID__c,Section__c,Start_Date__c,End_Date__c,ID_Student__c,ID_Class__c
0,25004961,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhwv,a014x000008VgET
1,25003514,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhx0,a014x000008VgET
2,25005833,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhx5,a014x000008VgET
3,25007334,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhzG,a014x000008VgET
4,25002589,GWU-ARL-DATA-PT-09-0,2020-09-16,None,a034x000002jhxA,a014x000008VgET


In [184]:
classparticipant = classpart_data_df_mg2[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]
#classparticipant['Start_Date__c'] = pd.to_datetime(classparticipant['Start_Date__c']).dt.date
classparticipant.head()

,ID_Student__c,ID_Class__c,Start_Date__c,End_Date__c
0,a034x000002jhwv,a014x000008VgET,2020-09-16,None
1,a034x000002jhx0,a014x000008VgET,2020-09-16,None
2,a034x000002jhx5,a014x000008VgET,2020-09-16,None
3,a034x000002jhzG,a014x000008VgET,2020-09-16,None
4,a034x000002jhxA,a014x000008VgET,2020-09-16,None


In [178]:
classparticipant_data_records = classparticipant.to_dict('records')
classparticipant_data_records

[{'ID_Student__c': 'a034x000002jhwv',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhx0',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhx5',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhzG',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhxA',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhxF',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None},
 {'ID_Student__c': 'a034x000002jhxK',
  'ID_Class__c': 'a014x000008VgET',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Da

In [187]:
for rec in classparticipant_data_records:

    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Student__c': rec['ID_Student__c'],
        'StartDate__c': str(rec['Start_Date__c']),
        'EndDate__c': rec['End_Date__c'],
    }
    
    try:
        sf.classparticipant__c.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [8]:
class_records = sf.query("SELECT Id FROM classparticipant__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

[{'Id': 'a054x0000010cJ1AAI'},
 {'Id': 'a054x0000010cJ6AAI'},
 {'Id': 'a054x0000010cJBAAY'},
 {'Id': 'a054x0000010cJGAAY'},
 {'Id': 'a054x0000010cJLAAY'},
 {'Id': 'a054x0000010cJQAAY'},
 {'Id': 'a054x0000010cJVAAY'},
 {'Id': 'a054x0000010cJaAAI'},
 {'Id': 'a054x0000010cJfAAI'},
 {'Id': 'a054x0000010cJkAAI'},
 {'Id': 'a054x0000010cJpAAI'},
 {'Id': 'a054x0000010cJuAAI'},
 {'Id': 'a054x0000010cJzAAI'},
 {'Id': 'a054x0000010cK4AAI'},
 {'Id': 'a054x0000010cK9AAI'},
 {'Id': 'a054x0000010cKEAAY'},
 {'Id': 'a054x0000010cKJAAY'},
 {'Id': 'a054x0000010cKOAAY'},
 {'Id': 'a054x0000010cKTAAY'},
 {'Id': 'a054x0000010cKYAAY'},
 {'Id': 'a054x0000010cKdAAI'},
 {'Id': 'a054x0000010cKiAAI'},
 {'Id': 'a054x0000010cKnAAI'},
 {'Id': 'a054x0000010cKsAAI'},
 {'Id': 'a054x0000010cKxAAI'},
 {'Id': 'a054x0000010cL2AAI'},
 {'Id': 'a054x0000010cL7AAI'},
 {'Id': 'a054x0000010cLCAAY'}]

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [9]:
for rec in recs_to_delete:
    try:
        sf.classparticipant__c.delete(rec['Id'])
    except Exception as e:
        print(e)